In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import shutil
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import _load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline


ORCA_TMP = '.orca_tmp'

In [5]:
smi = 'COC1CNC1'
# smi = 'CO'


# mol = _load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


Number of conformers generated: 2


In [6]:
draw3Dconfs(mol)

num conformers 2


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [33]:
### QM calculation by ORCA for each conformer

options = {
    'method': 'B3LYP def2-SVP OPT',
    'solvent': 'THF',
    'geom_maxiter': 100,
    'n_jobs': 8,
}

def gen_input(conf, options: dict):
    method, solvent = options['method'], options['solvent']
    geom_maxiter, n_jobs = options['geom_maxiter'], options['n_jobs']

    input_str =""
    input_str += f"!{method}\n"
    if solvent:
        input_str += (f"%cpcm\n"
        f"  smd true\n"
        f"  SMDsolvent \"{solvent}\"\n"
        f"  end\n")
    input_str += f"%geom MaxIter {geom_maxiter} end\n"
    input_str += f"%pal nprocs {n_jobs} end\n"
    input_str += gen_xyz_block(conf)
    return input_str

def gen_xyz_block(conf):
    mol = conf.GetOwningMol()
    xyz_block = "*xyz 0 1\n"
    for i in range(conf.GetNumAtoms()):
        symbol = mol.GetAtomWithIdx(i).GetSymbol()
        x, y, z = conf.GetAtomPosition(i)
        xyz_block += f"{symbol:3}{x:20.8f}{y:20.8f}{z:20.8f}\n"
    xyz_block += "*\n"
    return xyz_block


conf = mol.GetConformer()
print(gen_input(conf, options))

!B3LYP def2-SVP OPT
%cpcm
  smd true
  SMDsolvent "THF"
  end
%geom MaxIter 100 end
%pal nprocs 8 end
*xyz 0 1
C            2.33417390          0.28725551         -0.03477194
O            1.21564085         -0.49476746          0.30436796
C            0.06488221         -0.16348239         -0.41940648
C           -0.73537482          1.07536781          0.03876973
N           -1.94835162          0.25292121          0.06364632
C           -1.16882270         -0.98547054         -0.02625267
H            2.58398848          0.17771561         -1.09732021
H            3.16046391         -0.07610987          0.57274590
H            2.16241680          1.34861848          0.17824706
H            0.26859505         -0.16319938         -1.49814015
H           -0.80488044          1.91091202         -0.66018819
H           -0.39784607          1.43625430          1.01632367
H           -2.53070738          0.34927554          0.88775137
H           -1.00427296         -1.50967779          0.91

In [41]:
ORCA_PATH = "/opt/orca-5.0.3"
os.environ['PATH'] = f"{ORCA_PATH}:{os.environ.get('PATH')}"
os.environ['LD_LIBRARY_PATH'] = "/opt/orca-5.0.3/openmpi-4.1.1/lib:/opt/orca-5.0.3/orca:"
os.environ['PATH'] = f"/opt/orca-5.0.3/openmpi-4.1.1/bin:{os.environ['PATH']}"

In [40]:
os.environ['PATH']

'/opt/openmpi-4.1.1/bin:/opt/orca-5.0.3:/opt/openmpi-4.1.1/bin:/opt/openmpi-4.1.1/bin:/opt/orca-5.0.3:/opt/openmpi-4.1.1/bin:/opt/orca-5.0.3:/home/ergot/miniforge3/envs/linux_qm/bin:/home/ergot/miniforge3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Files (x86)/Common Files/Oracle/Java/javapath:/mnt/c/Program Files/PerkinElmerInformatics/ChemOffice/ChemScript/Lib:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0:/mnt/c/Windows/System32/OpenSSH:/mnt/c/Program Files/PuTTY:/mnt/c/Users/alexander.korolyov/AppData/Local/Programs/Python/Python311/Scripts:/mnt/c/Users/alexander.korolyov/AppData/Local/Programs/Python/Python311:/mnt/c/Users/alexander.korolyov/AppData/Local/Microsoft/WindowsApps:/mnt/c/Users/alexander.korolyov/AppData/Local/JetBrains/Toolbox/scripts:/mnt/c/Users/alexander.korolyov/AppData/Local/Programs/Git/cmd:/mnt/c/Users/alexand

In [42]:
def set_orca_env():
    ORCA_PATH = "/opt/orca-5.0.3"
    os.environ['PATH'] = f"{ORCA_PATH}:{os.environ.get('PATH')}"
    os.environ['LD_LIBRARY_PATH'] = f"{ORCA_PATH}:{os.environ.get('LD_LIBRARY_PATH', '')}"
    os.environ['PATH'] = f"/opt/openmpi-4.1.1/bin:{os.environ['PATH']}"
    os.environ['LD_LIBRARY_PATH'] = f"/opt/openmpi-4.1.1/lib:{os.environ['LD_LIBRARY_PATH']}"

# print(os.environ['PATH'])

saved_work_dir = os.getcwd()
tmp_path = _create_tmp_dir(ORCA_TMP)
os.chdir(tmp_path)

with open('input.inp', 'w') as f:
    input = gen_input(conf, options)
    f.write(input)

subprocess.run(["/opt/orca-5.0.3/orca/orca", "input.inp", "--use-hwthread-cpus"])

os.chdir(saved_work_dir)
shutil.rmtree(tmp_path)


                                 *****************
                                 * O   R   C   A *
                                 *****************

                                            #,                                       
                                            ###                                      
                                            ####                                     
                                            #####                                    
                                            ######                                   
                                           ########,                                 
                                     ,,################,,,,,                         
                               ,,#################################,,                 
                          ,,##########################################,,             
                       ,#########################################, ''##